In [ ]:
sharedinputpath = snakemake.params["sharedinputpath"]

In [ ]:
WDPA_Ia = snakemake.input.WDPA1a
WDPA_Ib = snakemake.input.WDPA1b
WDPA_II = snakemake.input.WDPA2
WDPA_III = snakemake.input.WDPA3
WDPA_IV = snakemake.input.WDPA4

In [ ]:
onshore_turbine = "Vestas_V112_3MW"
offshore_bottom_turbine = "oedb:V164"
offshore_floating_turbine = "oedb:V164"

In [ ]:
heightshp = snakemake.input.elevation

In [ ]:
degreeshp = snakemake.input.slope

In [ ]:
desired_regions = snakemake.params.aggregated_regions

In [ ]:
codes = (
    2,
    4,
    5,
    6,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
)

In [ ]:
wind_onshore_codes_no_buffer = (
    3,
    4,
    5,
    7,
    8,
    9,
    10,
    11,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
)

wind_onshore_codes_buffer = {1: 2000, 2: 1000, 6: 5000}

In [ ]:
# The year for which to calculate the weather data:
year = snakemake.wildcards.year

# Geodata files to use for selecting country onshore and offshore area:
geodata_files = {
    "onshore": snakemake.input.euroshape,
    "offshore_bottom": snakemake.input.eurooffshoreshape,
}

# The square outer boundaries of Europe to consider,
# because we have downloaded ERA5 for this extent:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 81

In [ ]:
weatherdata = sharedinputpath / f"weatherdata/europe_{str(year)}.nc"

In [ ]:
CORINE = snakemake.input.corine

In [ ]:
panel = "CSi"
orientation = "latitude_optimal"

In [ ]:
file_name = geodata_files["onshore"]

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)


import geopandas as gpd
import pandas as pd
import xarray as xr
from atlite.gis import ExclusionContainer
from shapely.geometry import Polygon

In [ ]:
atlite.__version__  # should be 0.2.4

In [ ]:
xr.__version__  # should be 0.18.2

In [ ]:
# should include the extra maybe progressbar code without time

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

# Desired regions


In [ ]:
desired_regions

## Shape level

To get the desired data at shape level, the first input is the shapefile itself.


In [ ]:
file_name

In [ ]:
europe = (
    gpd.read_file(file_name)
    .replace({"GB": "UK", "EL": "GR"})
    .rename(columns={"NUTS_ID": "index"})
    .loc[:, ["index", "geometry"]]
    .sort_values("index")
    .set_index("index")
    .loc[desired_regions]
)

In [ ]:
europe

In [ ]:
year

In [ ]:
rectx1

In [ ]:
rectx2

In [ ]:
recty1

In [ ]:
recty2

In [ ]:
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe = gpd.clip(europe, polygon)

In [ ]:
europe.plot(figsize=(15, 15))

# Cutout preparation


In [ ]:
offshore_to_ISO3166 = {
    "Albania": "AL",
    "Belgium": "BE",
    "Bulgaria": "BG",
    "Croatia": "HR",
    "Cyprus": "CY",
    "Denmark": "DK",
    "Estonia": "EE",
    "Finland": "FI",
    "France": "FR",
    "Germany": "DE",
    "Greece": "GR",
    "Ireland": "IE",
    "Italy": "IT",
    "Latvia": "LV",
    "Lithuania": "LT",
    "Malta": "MT",
    "Netherlands": "NL",
    "Poland": "PL",
    "Portugal": "PT",
    "Romania": "RO",
    "Spain": "ES",
    "Slovenia": "SI",
    "Slovakia": "SK",
    "Sweden": "SE",
    "United Kingdom": "UK",
}

In [ ]:
boundaries = []
for geodata_file_name, geodata_file_path in geodata_files.items():
    print(geodata_file_path)
    boundaries.append(
        gpd.read_file(geodata_file_path)
        .replace(offshore_to_ISO3166)
        .rename(columns={"id": "index"})
        .set_index("index")
        .filter(items=desired_regions, axis=0)
        .bounds
    )

boundaries = pd.concat(boundaries)

In [ ]:
boundaries

In [ ]:
boundaries = boundaries.groupby(lambda x: "bountry").agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
)

In [ ]:
boundaries["minx"] = boundaries["minx"] - 2
boundaries["miny"] = boundaries["miny"] - 2
boundaries["maxx"] = boundaries["maxx"] + 2
boundaries["maxy"] = boundaries["maxy"] + 2
boundaries

In [ ]:
weatherdata

In [ ]:
cutout = atlite.Cutout(path=weatherdata)

In [ ]:
cutout.prepared_features

In [ ]:
cutout.prepare()

In [ ]:
cutout = atlite.Cutout(
    path="../3_intermediate_data/intermediatecutout.nc",
    data=cutout.data.sel(
        x=slice(
            boundaries.loc["bountry", "minx"],
            boundaries.loc["bountry", "maxx"],
        ),
        y=slice(
            boundaries.loc["bountry", "miny"],
            boundaries.loc["bountry", "maxy"],
        ),
    ),
)

In [ ]:
cutout.prepare()

## Indices for gridcells


In [ ]:
gridcellnamingfunction = (
    lambda x: "x"
    + (x.x * 100).astype("int").astype("str")
    + "y"
    + (x.y * 100).astype("int").astype("str")
)

In [ ]:
if snakemake.wildcards.spatial == "grid":
    (
        cutout.grid.assign(gridcell=gridcellnamingfunction)
        .loc[:, ["gridcell"]]
        .to_csv(snakemake.output["indreg"], header=False, index=False)
    )
else:
    with open(snakemake.output["indreg"], "w"):
        pass

# Solar


## Areas


In [ ]:
panel

In [ ]:
orientation

In [ ]:
excluder_solar = ExclusionContainer()

In [ ]:
CORINE

In [ ]:
codes

In [ ]:
excluder_solar.add_raster(CORINE, codes=codes)

In [ ]:
WDPA_Ia

In [ ]:
WDPA_Ib

In [ ]:
WDPA_II

In [ ]:
WDPA_III

In [ ]:
WDPA_IV

In [ ]:
excluder_solar.add_raster(WDPA_Ia)
excluder_solar.add_raster(WDPA_Ib)
excluder_solar.add_raster(WDPA_II)
excluder_solar.add_raster(WDPA_III)
excluder_solar.add_raster(WDPA_IV)

In [ ]:
degreeshp

In [ ]:
slope = gpd.read_file(degreeshp).to_crs(excluder_solar.crs)

slope = slope[slope["gridcode"] == 1]

# use invert=True to exclude the areas that are to steep instead of
# excluding everything else
excluder_solar.add_geometry(slope.geometry, invert=True)

In [ ]:
heightshp

In [ ]:
height = gpd.read_file(heightshp).to_crs(excluder_solar.crs)

height = height[height["gridcode"] == 1]

excluder_solar.add_geometry(height.geometry, invert=True)

In [ ]:
def cutoff_raster(technology, output):
    if technology == "pv":
        cf = cutout.pv(panel=panel, orientation=orientation, capacity_factor=True)
        smallestincluded = snakemake.params.cutoffs["solar"]

    if technology == "onwind":
        cf = cutout.wind(turbine=onshore_turbine, capacity_factor=True)
        smallestincluded = snakemake.params.cutoffs["onwind"]

    if technology == "offwind":
        cf = cutout.wind(turbine=offshore_bottom_turbine, capacity_factor=True)
        smallestincluded = snakemake.params.cutoffs["offwind"]

    excluded = cf.where(cf.values >= smallestincluded, other=1)
    excluded = excluded.where(cf.values < smallestincluded, other=0)
    cf_exclusion = excluded.rio.write_crs(europe.crs)
    cf_exclusion.rio.to_raster(output)

In [ ]:
if snakemake.wildcards.spatial == "grid":
    for exclusion_tech_type in [
        "cf_exclusion_solar",
        "cf_exclusion_windon",
        "cf_exclusion_windoff",
    ]:
        with open(snakemake.output[exclusion_tech_type], "w"):
            pass

In [ ]:
low_cf = snakemake.output.cf_exclusion_solar
if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["solar"] != 0:
    cutoff_raster("pv", low_cf)
    excluder_solar.add_raster(low_cf)

In [ ]:
availability_matrix_solar = cutout.availabilitymatrix(
    europe, excluder_solar, nprocesses=snakemake.threads
)
availability_matrix_solar

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_solar = availability_matrix_solar.stack(spatial=["x", "y"]) * area

capacity_matrix_solar = capacity_matrix_solar.reindex(
    spatial=area.indexes.get("spatial")
)

capacity_matrix_solar

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasSolar = (
        capacity_matrix_solar.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Solar." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero,
        ## which will significantly reduce input file size.
        ## downside is that the indices need to be processed seperately
        ## then because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
if snakemake.wildcards.spatial == "region":
    highRESareasSolar = (
        capacity_matrix_solar.sum("spatial")
        .to_pandas()
        .reset_index()
        .assign(
            new_idx=lambda x: "Solar." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
highRESareasSolar

In [ ]:
(
    highRESareasSolar.round(1).to_csv(
        snakemake.output["areassolar"], header=False, sep=" "
    )
)

## Capacity factors


In [ ]:
if snakemake.wildcards.spatial == "grid":
    _, capacity_factor_solar = cutout.pv(
        panel=panel,
        orientation=orientation,
        capacity_factor=True,
    )
if snakemake.wildcards.spatial == "region":
    capacity_factor_solar = cutout.pv(
        panel=panel,
        orientation=orientation,
        matrix=capacity_matrix_solar,
        per_unit=True,
        capacity_factor=True,
        index=capacity_matrix_solar.index,
    )

capacity_factor_solar

# Wind


In [ ]:
excluder_wind_onshore = ExclusionContainer()

In [ ]:
wind_onshore_codes_no_buffer

In [ ]:
wind_onshore_codes_buffer

In [ ]:
degreeshp

In [ ]:
slope = gpd.read_file(degreeshp).to_crs(excluder_wind_onshore.crs)

slope = slope[slope["gridcode"] == 1]

excluder_wind_onshore.add_geometry(slope.geometry, invert=True)
# use invert=True to exclude the areas that are to steep instead of
# excluding everything else

In [ ]:
heightshp

In [ ]:
height = gpd.read_file(heightshp).to_crs(excluder_wind_onshore.crs)

height = height[height["gridcode"] == 1]

excluder_wind_onshore.add_geometry(height.geometry, invert=True)

In [ ]:
if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["onwind"] != 0:
    low_cf = snakemake.output.cf_exclusion_windon
    cutoff_raster("onwind", low_cf)
    excluder_wind_onshore.add_raster(low_cf)

In [ ]:
excluder_wind_onshore

In [ ]:
availability_matrix_wind_onshore = cutout.availabilitymatrix(
    europe, excluder_wind_onshore
)
availability_matrix_wind_onshore

In [ ]:
availability_matrix_wind_onshore.sum()

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_onshore = (
    availability_matrix_wind_onshore.stack(spatial=["x", "y"]) * area
)

capacity_matrix_wind_onshore = capacity_matrix_wind_onshore.reindex(
    spatial=area.indexes.get("spatial")
)

capacity_matrix_wind_onshore

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasWindOnshore = (
        capacity_matrix_wind_onshore.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Windonshore." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero, which will
        # significantly reduce input file size.
        ## downside is that the indices need to be processed seperately then
        # because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

if snakemake.wildcards.spatial == "region":
    highRESareasWindOnshore = (
        capacity_matrix_wind_onshore.sum("spatial")
        .to_pandas()
        .reset_index()
        .assign(
            new_idx=lambda x: "Windonshore." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
highRESareasWindOnshore

In [ ]:
(
    highRESareasWindOnshore.round(1).to_csv(
        snakemake.output["areaswindonshore"], header=False, sep=" "
    )
)

In [ ]:
onshore_turbine

In [ ]:
if snakemake.wildcards.spatial == "grid":
    _, capacity_factor_wind_onshore = cutout.wind(
        turbine=onshore_turbine, capacity_factor=True
    )

if snakemake.wildcards.spatial == "region":
    capacity_factor_wind_onshore = cutout.wind(
        turbine=onshore_turbine,
        matrix=capacity_matrix_wind_onshore,
        per_unit=True,
        capacity_factor=True,
        index=capacity_matrix_wind_onshore.index,
    )

## Offshore wind bottom


### Areas


In [ ]:
geodata_files["offshore_bottom"]

In [ ]:
europe_offshore_bottom = (
    gpd.read_file(geodata_files["offshore_bottom"])
    .replace(offshore_to_ISO3166)
    .set_index("index")
    .filter(items=desired_regions, axis=0)
)
europe_offshore_bottom

In [ ]:
europe_offshore_bottom.plot()

In [ ]:
year

In [ ]:
rectx1

In [ ]:
rectx2

In [ ]:
recty1

In [ ]:
recty2

In [ ]:
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe_offshore_bottom = gpd.clip(europe_offshore_bottom, polygon)

In [ ]:
europe_offshore_bottom.plot(figsize=(15, 15))

In [ ]:
excluder_wind_offshore_bottom = ExclusionContainer()

In [ ]:
excluder_wind_offshore_bottom

In [ ]:
WDPA_Ia

In [ ]:
WDPA_Ib

In [ ]:
WDPA_II

In [ ]:
WDPA_III

In [ ]:
WDPA_IV

In [ ]:
excluder_wind_offshore_bottom.add_raster(WDPA_Ia)
excluder_wind_offshore_bottom.add_raster(WDPA_Ib)
excluder_wind_offshore_bottom.add_raster(WDPA_II)
excluder_wind_offshore_bottom.add_raster(WDPA_III)
excluder_wind_offshore_bottom.add_raster(WDPA_IV)

In [ ]:
if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["offwind"] != 0:
    low_cf = snakemake.output.cf_exclusion_windoff
    cutoff_raster("offwind", low_cf)
    excluder_wind_offshore_bottom.add_raster(low_cf)

In [ ]:
availability_matrix_wind_offshore_bottom = cutout.availabilitymatrix(
    europe_offshore_bottom, excluder_wind_offshore_bottom, nprocesses=snakemake.threads
)
availability_matrix_wind_offshore_bottom

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_offshore_bottom = (
    availability_matrix_wind_offshore_bottom.stack(spatial=["x", "y"]) * area
)

capacity_matrix_wind_offshore_bottom = capacity_matrix_wind_offshore_bottom.reindex(
    spatial=area.indexes.get("spatial")
)

capacity_matrix_wind_offshore_bottom

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasWindOffshoreBottom = (
        capacity_matrix_wind_offshore_bottom.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Windoffshore." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero, which will
        # significantly reduce input file size.
        ## downside is that the indices need to be processed seperately then
        # because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

if snakemake.wildcards.spatial == "region":
    highRESareasWindOffshoreBottom = (
        capacity_matrix_wind_offshore_bottom.sum("spatial")
        .to_pandas()
        .reset_index()
        .assign(
            new_idx=lambda x: "Windoffshore." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
highRESareasWindOffshoreBottom

In [ ]:
highRESareasWindOffshoreBottom.round(1).to_csv(
    snakemake.output["areaswindoffshore"], header=False, sep=" "
)

In [ ]:
offshore_bottom_turbine

In [ ]:
if snakemake.wildcards.spatial == "grid":
    _, capacity_factor_wind_offshore_bottom = cutout.wind(
        turbine=offshore_bottom_turbine, capacity_factor=True
    )

if snakemake.wildcards.spatial == "region":
    capacity_factor_wind_offshore_bottom = cutout.wind(
        turbine=offshore_bottom_turbine,
        matrix=capacity_matrix_wind_offshore_bottom,
        per_unit=True,
        capacity_factor=True,
        index=capacity_matrix_wind_offshore_bottom.index,
    )

In [ ]:
capacity_factor_wind_offshore_bottom

# highRES-export-capacity_factors


Now we need to get it in the correct output format:


In [ ]:
capacity_factor_solar.name = None

In [ ]:
capacity_factor_wind_onshore.name = None

In [ ]:
capacity_factor_solar.name = "Solar"
capacity_factor_wind_onshore.name = "Windonshore"
capacity_factor_wind_offshore_bottom.name = "Windoffshore"

https://stackoverflow.com/questions/56457280/merging-xarray-files-into-a-new-dimension


In [ ]:
concatlist = [
    capacity_factor_solar,
    capacity_factor_wind_onshore,
    capacity_factor_wind_offshore_bottom,
]

In [ ]:
for i in concatlist:
    print(i.name)

In [ ]:
range(0, len(capacity_factor_solar.time))

In [ ]:
xr.concat(concatlist, pd.Index(list(da.name for da in concatlist), name="technology"))

In [ ]:
gridcellnamingfunctionxr = (
    lambda da: "x"
    + (da.x * 100).astype("int").astype("str").astype("object")
    + "y"
    + (da.y * 100).astype("int").astype("str").astype("object")
)

In [ ]:
year

In [ ]:
if snakemake.wildcards.spatial == "grid":
    (
        xr.concat(
            concatlist, pd.Index(list(da.name for da in concatlist), name="technology")
        )
        .stack(spatial=["x", "y"])
        .assign_coords(gridcell=gridcellnamingfunctionxr)
        .set_index(spatial="gridcell")
        .drop_vars(["lat", "lon"])
        .assign_coords(time=range(0, len(concatlist[0].time)))
        .stack(final=["time", "technology", "spatial"])
        .round(2)
        .to_pandas()
        .to_csv(snakemake.output["capfacfile"])
    )

if snakemake.wildcards.spatial == "region":
    (
        xr.concat(
            concatlist, pd.Index(list(da.name for da in concatlist), name="technology")
        )
        .assign_coords(time=range(0, len(concatlist[0].time)))
        .rename({"index": "spatial"})
        .stack(final=["time", "technology", "spatial"])
        .round(2)
        .to_pandas()
        .to_frame()
        .reset_index()
        .fillna(0)
        .set_index(["time", "technology", "spatial"])
        .to_csv(snakemake.output["capfacfile"])
    )